In [0]:
spark.conf.set("fs.azure.account.auth.type.streamingsa.dfs.core.windows.net", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type.streamingsa.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set("fs.azure.sas.fixed.token.streamingsa.dfs.core.windows.net", {token})

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
silver_schema = StructType([
    StructField("temperature", IntegerType()),
    StructField("humidity", IntegerType()),
    StructField("city", StringType()),
    StructField("timestamp", TimestampType())
])

In [0]:
df2 = (spark.readStream
      .format("parquet")
      .schema(silver_schema)
      .load("abfss://silver@streamingsa.dfs.core.windows.net/EventHub/weather/*.parquet")
      .withColumn("date", to_date("timestamp", "yyyy-MM-dd"))
      .withColumn("hour", hour("timestamp"))
)

(df2.writeStream
 .option("checkpointLocation", "abfss://checkpoint@streamingsa.dfs.core.windows.net/checkpoint2")
 .outputMode("append")
 .format("delta")
 .option("path", "abfss://gold@streamingsa.dfs.core.windows.net/EventHub/weather")
 .start()
 )